In [96]:
type Node = { 
    x: int; 
    y: int 
}
module Node =
    let Empty = { x = 0; y = 0 }
    let up node =
        { node with y = node.y + 1 } 
    let down node =
        { node with y = node.y - 1 } 
    let left node =
        { node with x = node.x - 1 }
    let right node =
        { node with x = node.x + 1 }

module Tail =
    let follow head tail =
        let xoffset = head.x - tail.x
        let yoffset = head.y - tail.y
        
        let adjustX =
            if xoffset > 0 then Node.right else Node.left

        let adjustY =
            if yoffset > 0 then Node.up else Node.down

        match Math.Abs(xoffset), Math.Abs(yoffset) with
        | x, y when x <= 1 && y <= 1 ->
            tail
        | x, y when x = 2 && y = 0 ->
            tail |> adjustX
        | x, y when x = 0 && y = 2 ->
            tail |> adjustY
        | _ -> tail |> adjustX |> adjustY

type Rope = Node seq
module Rope =
    let withSize size =
        [1..size] |> Seq.map (fun _-> Node.Empty)

    let move (direction:Node->Node) (rope:Rope) =
        let head = rope |> Seq.head |> direction
        let tail = 
            rope 
            |> Seq.skip 1
            |> Seq.mapFold (fun state node -> 
                let newTail = node |> Tail.follow state
                newTail,newTail
            ) head
            |> fst
        tail |> Seq.append [head]

let unpack (line:string) =
    match line.Split(" ") with 
    | [| dir; steps |] -> seq { for _ in 1..(steps |> int) do yield dir }
    | _ -> Seq.empty

let parser (move:string) (rope:Rope) =
    match move with
    | "U" -> rope |> Rope.move Node.up 
    | "D" -> rope |> Rope.move Node.down
    | "L" -> rope |> Rope.move Node.left 
    | "R" -> rope |> Rope.move Node.right
    | _ -> rope

let folder (rope:Rope) (line:string) =
    let newRope = rope |> parser line
    (newRope), newRope

let ResolutionFolder = __SOURCE_DIRECTORY__
let lines = File.ReadLines(ResolutionFolder + "/input9.txt")

let solve lines ropeSize =
    lines
    |> Seq.map unpack
    |> Seq.concat
    |> Seq.mapFold folder (Rope.withSize ropeSize)
    |> fst
    |> Seq.map Seq.last
    |> Set.ofSeq
    |> Seq.length

// part 1
solve lines 2 |> display

// part 2
solve lines 10 |> display

6337

2455